### 스마트폰 센서 데이터 실험

In [125]:
import pandas as pd
import os

In [126]:
source_dir = "origin_data/sensor"  # 원본 csv 파일 경로
dest_dir = "new"  # 35초 간격으로 처리한 후의 csv 파일 경로

In [127]:
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

In [128]:
name_list = ['anseonyeong','baekseungdo','cheonaeji','chunjihun','hurhongjun','jeongyubin','jojeongduk','kangjihyun','kangminjae','kimgangsu', 'kimminju', 'leeeunseo', 'leegahyeon', 'leegihun', 'leejaeho', 'leekanghyuk', 'leeseunglee', 'leeyunguel', 'ohseunghun', 'seosanghyeok']
course_list = ['A', 'B', 'C']
round_list = ['1', '2', '3', '4', '5', '6']

In [129]:
for name in name_list:
    for course in course_list:
        for round in round_list:
            file_path = f"{source_dir}/{name}/{course}/{round}/Accelerometer.csv"

            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                start_time = 0
                end_time = 35

                # 필요한 하위 디렉토리 생성
                dest_subdir = os.path.join(dest_dir, name, course, round)
                if not os.path.exists(dest_subdir):
                    os.makedirs(dest_subdir)

                while end_time <= df['seconds_elapsed'].max():
                    filtered_df = df[(df['seconds_elapsed'] >= start_time) & (df['seconds_elapsed'] <= end_time)]
                    file_name = os.path.join(dest_subdir, f"Filtered_Accelerometer_{start_time}-{end_time}.csv")
                    filtered_df.to_csv(file_name, index=False)
                    start_time = end_time
                    end_time += 35

In [130]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
from scipy.signal import spectrogram
import cv2
import os

In [131]:
source_dir = "new"  # 소스 디렉토리 경로
dest_dir = "longitudinal_spectrogram"  # 대상 디렉토리 경로

In [132]:
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

In [133]:
name_list = ['anseonyeong','baekseungdo','cheonaeji','chunjihun','hurhongjun','jeongyubin','jojeongduk','kangjihyun','kangminjae','kimgangsu', 'kimminju', 'leeeunseo', 'leegahyeon', 'leegihun', 'leejaeho', 'leekanghyuk', 'leeseunglee', 'leeyunguel', 'ohseunghun', 'seosanghyeok']
course_list = ['A','B','C']
round_list = ['1', '2', '3', '4', '5', '6']

In [134]:
for name in name_list:
    for course in course_list:
        for round in round_list:
            source_path = os.path.join(source_dir, name, course, round)
            dest_path = os.path.join(dest_dir, f"{name}_spectrogram", course, round)

            if not os.path.exists(dest_path):
                os.makedirs(dest_path)

            n = 0

            for file_name in os.listdir(source_path):
                if file_name.endswith(".csv"):
                    n += 1
                    # CSV 파일 로드
                    data = pd.read_csv(os.path.join(source_path, file_name))
                    time = data['seconds_elapsed'] - data['seconds_elapsed'].iloc[0]

                    longitudinal_acceleration = data['x']

                    f, t, Sxx = spectrogram(longitudinal_acceleration, fs=5, nperseg=224, noverlap=112)
                    Sxx = np.log(Sxx)
                    Sxx_normalized = (Sxx - Sxx.min()) / (Sxx.max() - Sxx.min())

                    colored_spectrogram = cm.viridis(Sxx_normalized)
                    colored_spectrogram_bgr = cv2.cvtColor((colored_spectrogram * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

                    filename = os.path.splitext(file_name)[0]
                    npy_save_path = os.path.join(dest_path, f"spectrogram_{n}.npy")
                    np.save(npy_save_path, np.array(colored_spectrogram_bgr))

                    n += 1

In [135]:
import os
import csv
import glob

In [136]:
label_mapping = {
    'anseonyeong':0,'baekseungdo':1,'cheonaeji':2,'chunjihun':3,'hurhongjun':4,'jeongyubin':5,'jojeongduk':6,'kangjihyun':7,'kangminjae':8,'kimgangsu':9, 'kimminju':10, 'leeeunseo':11, 'leegahyeon':12, 'leegihun':13, 'leejaeho':14, 'leekanghyuk':15, 'leeseunglee':16, 'leeyunguel':17, 'ohseunghun':18, 'seosanghyeok':19
}

In [137]:
course_mapping = {
    "A": 0,"B": 1,"C": 2
}

In [138]:
round_mapping = {
    "1": 0,"2": 1,"3": 2,"4": 3,"5": 4, "6":5
}

In [139]:
name_lt = ['anseonyeong','baekseungdo','cheonaeji','chunjihun','hurhongjun','jeongyubin','jojeongduk','kangjihyun','kangminjae','kimgangsu']
course_list = ['A','B','C']
round_lt = ['1', '2', '3', '4', '5', '6']

In [140]:
# CSV 파일 생성 및 열 이름 설정
csv_filename = "20_longitudinal_spectrogram.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["label", "course", "round", "file"])
    frame = 0
    for name in name_lt:

        for c in course:
            for r in round_lt:
                path = f'./longitudinal_spectrogram/{name}_spectrogram/{c}/{r}'
                file_list = os.listdir(path)  # 파일명

                for f in file_list:
                    frame += 1
                    file = os.path.join(path, f)

                    label = label_mapping.get(name, -1)  # Label 매핑
                    course_name = course_mapping.get(c, -1)  # Course 매핑
                    round = round_mapping.get(r, -1)
                    # print(label, course_name, file)
                    # CSV 파일에 데이터 쓰기
                    writer.writerow([label, course_name, r, file])
                print(f'{name}_{c} --> done')

print(f'CSV 파일 "{csv_filename}"이 생성되었습니다.')

anseonyeong_C --> done
anseonyeong_C --> done
anseonyeong_C --> done
anseonyeong_C --> done
anseonyeong_C --> done
anseonyeong_C --> done
baekseungdo_C --> done
baekseungdo_C --> done
baekseungdo_C --> done
baekseungdo_C --> done
baekseungdo_C --> done
baekseungdo_C --> done
cheonaeji_C --> done
cheonaeji_C --> done
cheonaeji_C --> done
cheonaeji_C --> done
cheonaeji_C --> done
cheonaeji_C --> done
chunjihun_C --> done
chunjihun_C --> done
chunjihun_C --> done
chunjihun_C --> done
chunjihun_C --> done
chunjihun_C --> done
hurhongjun_C --> done
hurhongjun_C --> done
hurhongjun_C --> done
hurhongjun_C --> done
hurhongjun_C --> done
hurhongjun_C --> done
jeongyubin_C --> done
jeongyubin_C --> done
jeongyubin_C --> done
jeongyubin_C --> done
jeongyubin_C --> done
jeongyubin_C --> done
jojeongduk_C --> done
jojeongduk_C --> done
jojeongduk_C --> done
jojeongduk_C --> done
jojeongduk_C --> done
jojeongduk_C --> done
kangjihyun_C --> done
kangjihyun_C --> done
kangjihyun_C --> done
kangjihyun

In [141]:
import numpy as np
import pandas as pd
import os
import cv2
import glob
import timm
from tqdm import tqdm
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

In [142]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [73]:
class CustomDataset(Dataset):
    def __init__(self, csv, mode):
        self.csv = csv
        self.mode = mode

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, idx):
        file = self.csv.loc[idx, 'file']
        label = torch.as_tensor(self.csv.loc[idx, 'label'], dtype=torch.long)
        file = np.load(file)

        return torch.Tensor(file).permute(2, 0, 1), label

In [74]:
class ResNet50GRUClassifier(nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers):
        super(ResNet50GRUClassifier, self).__init__()

        self.resnet = timm.create_model('resnet50', pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(2048, hidden_size, num_layers, batch_first=True, dropout=0.5)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        features = self.resnet(x)
        features = features.view(features.size(0), -1)

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(features.unsqueeze(1), h0)

        output = self.fc(out[:, -1, :])
        return output

In [75]:
def calculate_voting_3(model, dataloader):
    voting_3 = 0
    voting_3_t = 0

    for data, labels in dataloader:
        data, labels = data.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(data)
            ps = F.softmax(outputs, dim=1)

            top_p, top_class = ps.topk(1, dim=1)

        for i in range(len(labels)):
            label = labels[i]
            prediction = top_class[i].item()
            voting_3 += prediction == label
            voting_3_t += 1

    return (voting_3 / voting_3_t) * 100

In [76]:
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = False

set_seed(7777)

In [77]:
from sklearn.metrics import precision_recall_curve, auc

In [78]:
def calculate_ap(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

In [79]:
def calculate_map(y_true, y_scores, num_classes):
    aps = []
    for class_id in range(num_classes):
        class_targets = [1 if label == class_id else 0 for label in y_true]
        class_scores = [score[class_id] for score in y_scores]
        aps.append(calculate_ap(class_targets, class_scores))
    return sum(aps) / len(aps)

In [80]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [81]:
# Load your data CSV file
df = pd.read_csv('./10_longitudinal_spectrogram_10s.csv')

In [147]:
import numpy as np
import pandas as pd
import os
import cv2
import glob
import timm
from tqdm import tqdm
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

class CustomDataset(Dataset):
    def __init__(self, csv, mode):
        self.csv = csv
        self.mode = mode

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, idx):
        file = self.csv.loc[idx, 'file']
        label = torch.as_tensor(self.csv.loc[idx, 'label'], dtype=torch.long)
        file = np.load(file)

        return torch.Tensor(file).permute(2, 0, 1), label

class ResNet50GRUClassifier(nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers):
        super(ResNet50GRUClassifier, self).__init__()

        self.resnet = timm.create_model('resnet50', pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(2048, hidden_size, num_layers, batch_first=True, dropout=0.5)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        features = self.resnet(x)
        features = features.view(features.size(0), -1)

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(features.unsqueeze(1), h0)

        output = self.fc(out[:, -1, :])
        return output

def calculate_voting_3(model, dataloader):
    voting_3 = 0
    voting_3_t = 0

    for data, labels in dataloader:
        data, labels = data.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(data)
            ps = F.softmax(outputs, dim=1)

            top_p, top_class = ps.topk(1, dim=1)

        for i in range(len(labels)):
            label = labels[i]
            prediction = top_class[i].item()
            voting_3 += prediction == label
            voting_3_t += 1

    return (voting_3 / voting_3_t) * 100

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = False

set_seed(7777)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load your data CSV file
df = pd.read_csv('./20_longitudinal_spectrogram.csv')

n_fold = 6  # 회차별
# n_fold = 3 # 코스별

for fold in range(0, n_fold):
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold + 1}/{n_fold} ...')

    # Split the data into training and validation sets based on the fold
    df_train = df[df['round'] != fold + 1]
    df_valid = df[df['round'] == fold + 1]

    # df_train = df[df['course'] != fold]
    # df_valid = df[df['course'] == fold]

    df_train = df_train.reset_index()
    df_valid = df_valid.reset_index()

    batch_size = 10

    train_dataset = CustomDataset(df_train, 'train')
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = CustomDataset(df_valid, 'valid')
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    n_epochs = 15

    best_valid_loss = np.Inf
    best_valid_accuracy = 0.0

    train_loss = torch.zeros(n_epochs)
    valid_loss = torch.zeros(n_epochs)
    train_acc = torch.zeros(n_epochs)
    valid_acc = torch.zeros(n_epochs)

    # 모델 생성
    num_classes = 20
    hidden_size = 128
    num_layers = 2

    model = ResNet50GRUClassifier(num_classes, hidden_size, num_layers).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, min_lr=1e-8,
                                                     verbose=True)

    for e in range(0, n_epochs): # train
        print(f'epoch: {e}')

        train_preds = []
        train_probs = []
        train_labels = []

        model_preds = []
        model_labels = []
        model_probs = []

        model.to(device)
        model.train()

        for data, labels in tqdm(train_loader):
            labels = labels.type(torch.LongTensor)
            data, labels = data.to(device), labels.to(device)

            outputs = model(data)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss[e] += loss.item()

            ps = F.softmax(outputs, dim=1)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.reshape(top_class.shape)
            train_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()

        train_loss[e] /= len(train_loader)
        train_acc[e] /= len(train_loader)

        with torch.no_grad(): # validation
            model.eval()

            val_loss = []
            mse_loss = []
            weight_loss = []

            logit_list = []
            target_list = []

            # class 맞춘 개수
            class_correct = torch.zeros(10)
            # 전체 data를 class 별 갯수로 정리
            class_total = torch.zeros(10)

            for data, labels in tqdm(valid_loader):
                labels = labels.type(torch.LongTensor)
                data, labels = data.to(device), labels.to(device)

                logits = model(data)
                loss = criterion(logits, labels)

                valid_loss[e] += loss.item()
                val_loss.append(loss.detach().cpu().numpy())

                ps = F.softmax(logits, dim=1)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.reshape(top_class.shape)
                valid_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()

                ps = ps.detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()

                for i in range(len(labels)):
                    label = labels[i]
                    class_correct[label] += equals[i].item()
                    class_total[label] += 1

                    logit_list.append(ps[i])
                    target_list.append(labels[i])


            voting_1 = 0
            voting_1_t = 0
            voting_3 = 0
            voting_3_t = 0
            voting_10 = 0
            voting_10_t = 0
            vote_all = 0
            vote_all_t = 0

            voting_1_top3 = 0
            voting_1_top3_t = 0
            voting_3_top3 = 0
            voting_3_top3_t = 0
            voting_10_top3 = 0
            voting_10_top3_t = 0
            voting_all_top3 = 0
            voting_all_top3_t = 0

            voting_1_TP_total = 0
            voting_1_FP_total = 0
            voting_1_FN_total = 0
            voting_3_TP_total = 0
            voting_3_FP_total = 0
            voting_3_FN_total = 0
            voting_10_TP_total = 0
            voting_10_FP_total = 0
            voting_10_FN_total = 0
            voting_all_TP_total = 0
            voting_all_FP_total = 0
            voting_all_FN_total = 0

            y_true_1, y_scores_1 = [], []
            y_true_3, y_scores_3 = [], []
            y_true_10, y_scores_10 = [], []
            y_true_all, y_scores_all = [], []

            # 사람 위치 찾기
            for i in range(len(ps[0])):
                people_index = list(filter(lambda x: target_list[x] == i, range(len(target_list))))

                if len(people_index) == 0:
                    continue  # 해당 레이블과 일치하는 데이터가 없는 경우 다음 반복으로 건너뜁니다.

                softmax = sum([logit_list[x] for x in people_index]) / len(people_index)
                vote_all += np.argmax(softmax) == i
                vote_all_t += 1

                top_3_indices = np.argsort(softmax)[-3:]
                voting_all_top3 += i in top_3_indices
                voting_all_top3_t += 1

                pred_label = np.argmax(softmax)



                # F1 점수를 위한 카운팅
                if pred_label == i:
                    voting_all_TP_total += 1
                else:
                    voting_all_FP_total += 1
                    voting_all_FN_total += 1

                y_scores_all.append(softmax[i])
                y_true_all.append(pred_label == i)

                for x in range(0, len(people_index), 1):
                    softmax_1 = sum([logit_list[j] for j in people_index[x:x + 1]])
                    voting_1 += np.argmax(softmax_1) == i
                    voting_1_t += 1

                    top_3_indices = np.argsort(softmax_1)[-3:]
                    voting_1_top3 += i in top_3_indices
                    voting_1_top3_t += 1

                    pred_label = np.argmax(softmax_1)

                    # F1 점수를 위한 카운팅
                    if pred_label == i:
                        voting_1_TP_total += 1
                    else:
                        voting_1_FP_total += 1
                        voting_1_FN_total += 1

                    y_scores_1.append(softmax_1[i])
                    y_true_1.append(pred_label == i)

                for x in range(0, len(people_index), 3):
                    softmax_3 = sum([logit_list[j] for j in people_index[x:x + 3]])
                    voting_3 += np.argmax(softmax_3) == i
                    voting_3_t += 1

                    top_3_indices = np.argsort(softmax_3)[-3:]
                    voting_3_top3 += i in top_3_indices
                    voting_3_top3_t += 1

                    pred_label = np.argmax(softmax_3)

                    # F1 점수를 위한 카운팅
                    if pred_label == i:
                        voting_3_TP_total += 1
                    else:
                        voting_3_FP_total += 1
                        voting_3_FN_total += 1

                    y_scores_3.append(softmax_3[i])
                    y_true_3.append(pred_label == i)

                for x in range(0, len(people_index), 10):
                    softmax_10 = sum([logit_list[j] for j in people_index[x:x + 10]])
                    voting_10 += np.argmax(softmax_10) == i
                    voting_10_t += 1

                    top_3_indices = np.argsort(softmax_10)[-3:]
                    voting_10_top3 += i in top_3_indices
                    voting_10_top3_t += 1

                    pred_label = np.argmax(softmax_10)

                    # F1 점수를 위한 카운팅
                    if pred_label == i:
                        voting_10_TP_total += 1
                    else:
                        voting_10_FP_total += 1
                        voting_10_FN_total += 1

                    y_scores_10.append(softmax_10[i])
                    y_true_10.append(pred_label == i)

        valid_loss[e] /= len(valid_loader)
        valid_acc[e] /= len(valid_loader)

        print('Training Loss: {:.6f} \tValidation Loss: {:.6f}'.format(train_loss[e], valid_loss[e]))
        print('Train Accuracy: {:.6f} \tValid Accuracy: {:.6f}'.format(train_acc[e], valid_acc[e]))

        if valid_loss[e] <= best_valid_loss:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(best_valid_loss,
                                                                                            valid_loss[e]))
            torch.save(model.state_dict(), f'./model/{fold+1}round_20person.pt')
            best_valid_loss = valid_loss[e]

        model.load_state_dict(torch.load(f'./model/{fold+1}round_20person.pt'))

        # After training for each fold, you can calculate the voting_3 and voting_10 for that fold:
        voting_1_accuracy = (voting_1/voting_1_t)
        voting_3_accuracy = (voting_3 / voting_3_t)
        voting_10_accuracy = (voting_10 / voting_10_t)
        voting_all_accuracy = (vote_all / vote_all_t)

        print(f"voting_1_ ACC: {voting_1_accuracy}")
        print(f"voting_3_ ACC: {voting_3_accuracy}")
        print(f"voting_10_ ACC: {voting_10_accuracy}")
        print(f"voting_all_ ACC: {voting_all_accuracy}")

        # 전체 F1 점수 계산
        voting_1_precision_total = voting_1_TP_total / (voting_1_TP_total + voting_1_FP_total) if (voting_1_TP_total + voting_1_FP_total) > 0 else 0
        voting_1_recall_total = voting_1_TP_total / (voting_1_TP_total + voting_1_FN_total) if (voting_1_TP_total + voting_1_FN_total) > 0 else 0
        voting_1_f1_total = 2 * (voting_1_precision_total * voting_1_recall_total) / (voting_1_precision_total + voting_1_recall_total) if (voting_1_precision_total + voting_1_recall_total) > 0 else 0

        voting_3_precision_total = voting_3_TP_total / (voting_3_TP_total + voting_3_FP_total) if (voting_3_TP_total + voting_3_FP_total) > 0 else 0
        voting_3_recall_total = voting_3_TP_total / (voting_3_TP_total + voting_3_FN_total) if (voting_3_TP_total + voting_3_FN_total) > 0 else 0
        voting_3_f1_total = 2 * (voting_3_precision_total * voting_3_recall_total) / (voting_3_precision_total + voting_3_recall_total) if (voting_3_precision_total + voting_3_recall_total) > 0 else 0

        voting_10_precision_total = voting_10_TP_total / (voting_10_TP_total + voting_10_FP_total) if (voting_10_TP_total + voting_10_FP_total) > 0 else 0
        voting_10_recall_total = voting_10_TP_total / (voting_10_TP_total + voting_10_FN_total) if (voting_10_TP_total + voting_10_FN_total) > 0 else 0
        voting_10_f1_total = 2 * (voting_10_precision_total * voting_10_recall_total) / (voting_10_precision_total + voting_10_recall_total) if (voting_10_precision_total + voting_10_recall_total) > 0 else 0

        voting_all_precision_total = voting_all_TP_total / (voting_all_TP_total + voting_all_FP_total) if (voting_all_TP_total + voting_all_FP_total) > 0 else 0
        voting_all_recall_total = voting_all_TP_total / (voting_all_TP_total + voting_all_FN_total) if (voting_all_TP_total + voting_all_FN_total) > 0 else 0
        voting_all_f1_total = 2 * (voting_all_precision_total * voting_all_recall_total) / (voting_all_precision_total + voting_all_recall_total) if (voting_all_precision_total + voting_all_recall_total) > 0 else 0

        mAP_1 = average_precision_score(y_true_1, y_scores_1)
        print('voting_1_ MAP : ',mAP_1)

        mAP_3 = average_precision_score(y_true_3, y_scores_3)
        print('voting_3_ MAP : ', mAP_3)

        mAP_10 = average_precision_score(y_true_10, y_scores_10)
        print('voting_10_ MAP : ', mAP_10)

        mAP_all = average_precision_score(y_true_all, y_scores_all)
        print('voting_all_ MAP : ', mAP_all)

        print(f"voting_1_ F1 Score: {voting_1_f1_total}")
        print(f"voting_3_ F1 Score: {voting_3_f1_total}")
        print(f"voting_10_ F1 Score: {voting_10_f1_total}")
        print(f"voting_all_ F1 Score: {voting_all_f1_total}")

        # print(f'Voting_1 Accuracy for fold {fold + 1}: {voting_1_accuracy}%')
        print('voting_1 top3: ', voting_1_top3/voting_1_top3_t)
        # print(f'Voting_3 Accuracy for fold {fold + 1}: {voting_3_accuracy}%')
        print('voting_3 top3: ', voting_3_top3 / voting_3_top3_t)
        # print(f'Voting_10 Accuracy for fold {fold + 1}: {voting_10_accuracy}%')
        print('voting_10 top3: ', voting_10_top3 / voting_10_top3_t)
        # print(f'Voting_all Accuracy for fold {fold + 1}: {voting_all_accuracy}%')
        print('voting_all top3: ', voting_all_top3 / voting_all_top3_t)


cuda
cuda
------------------------------------------------------------------------
Training for fold 1/6 ...
epoch: 0


100%|██████████| 18/18 [00:00<00:00, 137.38it/s]


Training Loss: 2.513109 	Validation Loss: 2.344129
Train Accuracy: 0.112941 	Valid Accuracy: 0.100000
Validation loss decreased (inf --> 2.344129).  Saving model ...
voting_1_ ACC: 0.1
voting_3_ ACC: 0.09523809523809523
voting_10_ ACC: 0.09090909090909091
voting_all_ ACC: 0.1
voting_1_ MAP :  0.9621723529769508
voting_3_ MAP :  1.0
voting_10_ MAP :  0.7
voting_all_ MAP :  1.0
voting_1_ F1 Score: 0.10000000000000002
voting_3_ F1 Score: 0.09523809523809523
voting_10_ F1 Score: 0.09090909090909091
voting_all_ F1 Score: 0.10000000000000002
voting_1 top3:  0.3
voting_3 top3:  0.30158730158730157
voting_10 top3:  0.2727272727272727
voting_all top3:  0.3
epoch: 1


100%|██████████| 18/18 [00:00<00:00, 135.17it/s]


Training Loss: 2.324277 	Validation Loss: 2.310890
Train Accuracy: 0.121765 	Valid Accuracy: 0.133333
Validation loss decreased (2.344129 --> 2.310890).  Saving model ...
voting_1_ ACC: 0.13333333333333333
voting_3_ ACC: 0.14285714285714285
voting_10_ ACC: 0.09090909090909091
voting_all_ ACC: 0.1
voting_1_ MAP :  0.8322307956720988
voting_3_ MAP :  0.8162509924805008
voting_10_ MAP :  0.75
voting_all_ MAP :  1.0
voting_1_ F1 Score: 0.13333333333333333
voting_3_ F1 Score: 0.14285714285714285
voting_10_ F1 Score: 0.09090909090909091
voting_all_ F1 Score: 0.10000000000000002
voting_1 top3:  0.35555555555555557
voting_3 top3:  0.3492063492063492
voting_10 top3:  0.3181818181818182
voting_all top3:  0.3
epoch: 2


...

Training Loss: 2.076787 	Validation Loss: 2.260968
Train Accuracy: 0.220930 	Valid Accuracy: 0.170588
voting_1_ ACC: 0.17261904761904762
voting_3_ ACC: 0.1896551724137931
voting_10_ ACC: 0.19047619047619047
voting_all_ ACC: 0.2
voting_1_ MAP :  0.8898918819433649
voting_3_ MAP :  0.8078410159749873
voting_10_ MAP :  0.5520833333333333
voting_all_ MAP :  1.0
voting_1_ F1 Score: 0.17261904761904762
voting_3_ F1 Score: 0.1896551724137931
voting_10_ F1 Score: 0.19047619047619047
voting_all_ F1 Score: 0.20000000000000004
voting_1 top3:  0.4107142857142857
voting_3 top3:  0.46551724137931033
voting_10 top3:  0.5714285714285714
voting_all top3:  0.6
epoch: 13


100%|██████████| 17/17 [00:00<00:00, 137.66it/s]


Training Loss: 2.093461 	Validation Loss: 2.256172
Train Accuracy: 0.229845 	Valid Accuracy: 0.186765
voting_1_ ACC: 0.18452380952380953
voting_3_ ACC: 0.15517241379310345
voting_10_ ACC: 0.3333333333333333
voting_all_ ACC: 0.3
voting_1_ MAP :  0.8687884718671883
voting_3_ MAP :  0.8755199049316698
voting_10_ MAP :  0.6717592300077926
voting_all_ MAP :  0.8055555555555556
voting_1_ F1 Score: 0.18452380952380953
voting_3_ F1 Score: 0.15517241379310345
voting_10_ F1 Score: 0.3333333333333333
voting_all_ F1 Score: 0.3
voting_1 top3:  0.42857142857142855
voting_3 top3:  0.5172413793103449
voting_10 top3:  0.5238095238095238
voting_all top3:  0.6
epoch: 14


100%|██████████| 17/17 [00:00<00:00, 133.92it/s]


Training Loss: 2.065935 	Validation Loss: 2.313430
Train Accuracy: 0.231783 	Valid Accuracy: 0.155882
voting_1_ ACC: 0.15476190476190477
voting_3_ ACC: 0.13793103448275862
voting_10_ ACC: 0.09523809523809523
voting_all_ ACC: 0.1
voting_1_ MAP :  0.7345381556869671
voting_3_ MAP :  0.6886228354978354
voting_10_ MAP :  0.8333333333333333
voting_all_ MAP :  1.0
voting_1_ F1 Score: 0.15476190476190477
voting_3_ F1 Score: 0.13793103448275862
voting_10_ F1 Score: 0.09523809523809523
voting_all_ F1 Score: 0.10000000000000002
voting_1 top3:  0.3869047619047619
voting_3 top3:  0.3793103448275862
voting_10 top3:  0.42857142857142855
voting_all top3:  0.4
